# Feature Engineering

In [1]:
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


# Table of Contents
- [Dataset Preparation](#Dataset-Preparation)
  - [Load Data](#load-data)
  - [Validate Data](#validate-data)
- [Baseline Model](#Baseline-Model)
- [Baseline Model With Lagged Target Features](#baseline-model-with-lagged-target-features)
- [Add More Features](#add-more-features)
- [Train With TimeSeriesSplit](#train-with-timeseriessplit)
- [Tune Hyperparameters Using Optuna](#tune-hyperparameters-using-optuna)
- [Conclusions](#Conclusions)

In [4]:
from src.utilities.data_validator import data_validator

# Dataset Preparation

### Load data

In [5]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")

data.head()

Shape: (13903, 16)

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
"""2011-01-01 00:00:00""",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
"""2011-01-01 01:00:00""",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40
"""2011-01-01 02:00:00""",1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0.0,5,27,32
"""2011-01-01 03:00:00""",1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
"""2011-01-01 04:00:00""",1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Validate Data

In [6]:
# Validate the data
data_report: dict[str, Any] = data_validator(data)
console.print(data_report)

{
    'data_schema': {
        'numeric': {
            'season': 'Int64',
            'yr': 'Int64',
            'mnth': 'Int64',
            'hr': 'Int64',
            'holiday': 'Int64',
            'weekday': 'Int64',
            'workingday': 'Int64',
            'weathersit': 'Int64',
            'temp': 'Float64',
            'atemp': 'Float64',
            'hum': 'Float64',
            'windspeed': 'Float64',
            'casual': 'Int64',
            'registered': 'Int64',
            'cnt': 'Int64'
        },
        'string': {'datetime': 'String'}
    },
    'data_shape': {
        'total_rows': 13903,
        'total_columns': 16,
        'number_of_numeric_columns': 15,
        'number_of_string_columns': 1
    },
    'summary_statistics': {
        'numeric': [
            {
                'column': 'season',
                'mean': 2.26,
                'median': 2.0,
                'mode': [2],
                'std': 1.04,
                'variance': 1.07,
                'range': 3,
                'min': 1,
                'max': 4,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 4
            },
            {
                'column': 'yr',
                'mean': 0.38,
                'median': 0.0,
                'mode': [0],
                'std': 0.48,
                'variance': 0.24,
                'range': 1,
                'min': 0,
                'max': 1,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 2
            },
            {
                'column': 'mnth',
                'mean': 5.65,
                'median': 5.0,
                'mode': [7, 5],
                'std': 3.22,
                'variance': 10.36,
                'range': 11,
                'min': 1,
                'max': 12,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 12
            },
            {
                'column': 'hr',
                'mean': 11.55,
                'median': 12.0,
                'mode': [16, 17],
                'std': 6.91,
                'variance': 47.78,
                'range': 23,
                'min': 0,
                'max': 23,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 24
            },
            {
                'column': 'holiday',
                'mean': 0.03,
                'median': 0.0,
                'mode': [0],
                'std': 0.16,
                'variance': 0.03,
                'range': 1,
                'min': 0,
                'max': 1,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 2
            },
            {
                'column': 'weekday',
                'mean': 3.0,
                'median': 3.0,
                'mode': [6],
                'std': 2.01,
                'variance': 4.02,
                'range': 6,
                'min': 0,
                'max': 6,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 7
            },
            {
                'column': 'workingday',
                'mean': 0.68,
                'median': 1.0,
                'mode': [1],
                'std': 0.46,
                'variance': 0.22,
                'range': 1,
                'min': 0,
                'max': 1,
                'count': 13903,
                'missing_values': 0,
                'missing_pct': 0.0,
                'unique_values': 2
            },
            {
                'column': 'weathersit',
                'mean': 1.42,
                'median': 1.0,
                'mod

In [7]:
data.head()

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
"""2011-01-01 00:00:00""",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
"""2011-01-01 01:00:00""",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40
"""2011-01-01 02:00:00""",1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0.0,5,27,32
"""2011-01-01 03:00:00""",1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
"""2011-01-01 04:00:00""",1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Comment

- Drop the columns with high cardinality
    - `datetime`

- Drop irrelevant columns
    - `casual` (used to create the target)
    - `registered` (used to create the target)
    - `atemp` (highly correlated with `temp`) 
    - `yr` (not relevant)


In [8]:
year: dict[str, Any] = data_report.get("summary_statistics").get("numeric")[1]
atemp: dict[str, Any] = data_report.get("summary_statistics").get("numeric")[9]
casual: dict[str, Any] = data_report.get("summary_statistics").get("numeric")[-2]
registered: dict[str, Any] = data_report.get("summary_statistics").get("numeric")[-1]
datetime: dict[str, Any] = data_report.get("summary_statistics").get("categorical")[0]

console.print(
    f"Numeric:\n{year}\n\n{atemp}\n\n{casual}\n\n{registered}\n\nCategorical: \n{datetime}"
)

Numeric:
{'column': 'yr', 'mean': 0.38, 'median': 0.0, 'mode': [0], 'std': 0.48, 'variance': 0.24, 'range': 1, 'min': 0, 
'max': 1, 'count': 13903, 'missing_values': 0, 'missing_pct': 0.0, 'unique_values': 2}

{'column': 'atemp', 'mean': 0.48, 'median': 0.48, 'mode': [0.6212], 'std': 0.18, 'variance': 0.03, 'range': 1.0, 
'min': 0.0, 'max': 1.0, 'count': 13903, 'missing_values': 0, 'missing_pct': 0.0, 'unique_values': 65}

{'column': 'registered', 'mean': 140.6, 'median': 107.0, 'mode': [4], 'std': 137.51, 'variance': 18909.65, 'range':
796, 'min': 0, 'max': 796, 'count': 13903, 'missing_values': 0, 'missing_pct': 0.0, 'unique_values': 701}

{'column': 'cnt', 'mean': 174.64, 'median': 130.0, 'mode': [5], 'std': 166.96, 'variance': 27874.11, 'range': 956, 
'min': 1, 'max': 957, 'count': 13903, 'missing_values': 0, 'missing_pct': 0.0, 'unique_values': 789}

Categorical: 
{'column': 'datetime', 'total_count': 13903, 'unique_values': 13903, 'value_counts': [['2011-01-01 00:00:00', 1], 
['2011-01-01 01:00:00', 1], ['2011-01-01 02:00:00', 1], ['2011-01-01 03:00:00', 1], ['2011-01-01 04:00:00', 1], 
['2011-01-01 05:00:00', 1], ['2011-01-01 06:00:00', 1], ['2011-01-01 07:00:00', 1], ['2011-01-01 08:00:00', 1], 
['2011-01-01 09:00:00', 1]], 'missing_values': 0, 'missing_pct': 0.0}

In [9]:
columns_to_drop: list[str] = ["yr", "atemp", "casual", "registered", "datetime"]
df: pl.DataFrame = data.clone().drop(columns_to_drop)

df.head(3)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64
1,1,0,0,6,0,1,0.24,0.81,0.0,16
1,1,1,0,6,0,1,0.22,0.8,0.0,40
1,1,2,0,6,0,1,0.22,0.8,0.0,32


In [10]:
# Since we want to maintain the temporal order, we'll
# use a custom function for splitting the data

# it's assumed the data has been sorted in ascending order of time
# get the train, val, test ratios
# -10% of the training data is used for validation
train_size: float = 0.9
data_array: np.ndarray = df.to_numpy()
train_array: np.ndarray = data_array[: int(train_size * data_array.shape[0])]
test_array: np.ndarray = data_array[train_array.shape[0] :]

train_array.shape, test_array.shape
# convert the ratios to int values and select a slice of the data corresponding to the ratio

((12512, 11), (1391, 11))

In [11]:
from sklearn.metrics._regression import mean_absolute_error, root_mean_squared_error


def split_temporal_data(
    data: pl.DataFrame, test_size: float = 0.2
) -> tuple[pl.DataFrame, pl.DataFrame]:
    """Split data into training and testing sets while maintaining temporal order."""
    feature_names: list[str] = data.columns
    data_array: np.ndarray = data.to_numpy()
    train_size: float = int((1 - test_size) * data_array.shape[0])

    train_array: np.ndarray = data_array[:train_size]
    test_array: np.ndarray = data_array[train_size:]

    return pl.DataFrame(train_array, schema=feature_names), pl.DataFrame(
        test_array, schema=feature_names
    )


def compute_metrics(
    y_true: np.ndarray | list, y_pred: np.ndarray | list
) -> dict[str, float]:
    """
    Compute evaluation metrics between true and predicted values.

    Metrics returned:
    - MAPE: Mean Absolute Percentage Error (in %)
    - MAE: Mean Absolute Error
    - RMSE: Root Mean Squared Error

    Parameters:
    ----------
    y_true : array-like
        Ground truth values.
    y_pred : array-like
        Predicted values.

    Returns:
    -------
    dict
        Dictionary with keys 'MAPE', 'MAE', and 'RMSE' and their float values.
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    mape: float = (
        np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 0.01, y_true))) * 100
    ).item()

    return {
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "MAPE": round(mape, 2),
    }

### Target Definition

- We'll use the next hour `cnt` as the target variable to predict bike rentals.
- This is because bike rental services typically operate on an hourly basis, and predicting the next hour's demand can help in resource allocation and planning.
- We'll shift the `cnt` column by one hour to create the target variable, ensuring that our model learns to predict future demand based on current and past data.
- i.e. `df["target"] = df["cnt"].shift(-1)` if `cnt`=[5, 10, 15, 20] then `target`=[10, 15, 20, NaN]
  - i.e. instead of predicting the current hour's demand, we are training the model to predict the demand for the next hour based on the current and past data.

In [12]:
df = df.with_columns(pl.col("cnt").shift(-1).alias("target")).with_columns(
    pl.col("target").fill_null(strategy="forward")
)
display(df.null_count())
df.head(3)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0


season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13


In [13]:
df = df.drop("cnt")

train_df, test_df = split_temporal_data(df)

x_train = train_df.drop("target").to_numpy()
y_train = train_df["target"].to_numpy()

x_test = test_df.drop("target").to_numpy()
y_test = test_df["target"].to_numpy()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((11122, 10), (11122,), (2781, 10), (2781,))

<br>

# Baseline Model

- Create a simple and basic ML model to be used as reference.
- I used a `RandomForestRegressor` because it requires little tuning.

In [14]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(random_state=123)
rf_reg.fit(x_train, y_train)
rf_reg.score(x_test, y_test)

y_pred = rf_reg.predict(x_test)

In [15]:
metrics_base = compute_metrics(y_test, y_pred)
metrics_base

{'MAE': 84.78, 'RMSE': 116.41, 'MAPE': 34.11}

### Comment

- The model tends to underpredict during most of the high-peak periods.
- This suggests that the model is underestimating the number of bike rentals.

In [16]:
import plotly.graph_objects as go


def plot_actual_vs_predicted(
    y_true: np.ndarray | list,
    y_pred: np.ndarray | list,
    n: int = 300,
    title: str = "Actual vs Predicted",
) -> None:
    """Plot actual vs predicted values using Plotly."""
    print(f"Using first {n} values for plotting.")

    fig = go.Figure()

    # Add actual values trace
    fig.add_trace(
        go.Scatter(
            x=list(range(n)),
            y=y_true[:n],
            mode="lines",
            name="y_true (Actual)",
            line={"color": "blue", "width": 2},
        )
    )

    # Add predicted values trace
    fig.add_trace(
        go.Scatter(
            x=list(range(n)),
            y=y_pred[:n],
            mode="lines",
            name="y_pred (Predicted)",
            line={"color": "red", "width": 2},
        )
    )

    # Update layout for better visualization
    fig.update_layout(
        title=f"<b>{title}</b>",
        xaxis_title="Time Steps",
        yaxis_title="Bike Rental Count",
        hovermode="x unified",
        template="plotly_white",
        width=800,
        height=500,
    )

    # Show the interactive plot
    fig.show()

In [17]:
n: int = 300
plot_actual_vs_predicted(
    y_true=y_test,
    y_pred=y_pred,
    n=n,
    title="Actual vs Predicted Bike Rental Counts (Base Model)",
)

Using first 300 values for plotting.


### Dummy Model

- Let's create a dummy model that predicts the next hour's bike rentals based on exactly the same hour of the previous day.

In [18]:
y_dummy = pl.Series(y_test).shift(1).fill_null(strategy="backward").to_numpy()
y_dummy

array([463., 463., 738., ..., 327., 250., 214.], shape=(2781,))

In [19]:
metrics_dummy: dict[str, float] = compute_metrics(y_test, y_dummy)

console.print(f"Metrics (Base Model): {metrics_base}", style="info")
console.print(f"Metrics (Dummy Model): {metrics_dummy}", style="info")

Metrics (Base Model): {'MAE': 84.78, 'RMSE': 116.41, 'MAPE': 34.11}

Metrics (Dummy Model): {'MAE': 87.01, 'RMSE': 128.17, 'MAPE': 55.4}

### Comment

- The baseline model's performance is not a significant improvement over the dummy model. 

- While the Mean Absolute Percentage Error (`MAPE`) improved notably, the Mean Absolute Error (`MAE`) and other metrics show little change, suggesting that feature engineering is necessary to improve the model.

In [20]:
def pct_change(old: float, new: float) -> float:
    """Calculate the percentage change from old to new value."""
    if old == 0:
        return float("inf")  # Avoid division by zero
    return ((new - old) / old) * 100


mae_base, mae_dummy = metrics_base.get("MAE"), metrics_dummy.get("MAE")
rmse_base, rmse_dummy = metrics_base.get("RMSE"), metrics_dummy.get("RMSE")
mape_base, mape_dummy = metrics_base.get("MAPE"), metrics_dummy.get("MAPE")

console.print(
    f"MAE Change: {pct_change(old=mae_dummy, new=mae_base):.2f}%", style="highlight"
)
console.print(
    f"RMSE Change: {pct_change(old=rmse_dummy, new=rmse_base):.2f}%", style="highlight"
)
console.print(
    f"MAPE Change: {pct_change(old=mape_dummy, new=mape_base):.2f}%", style="highlight"
)

MAE Change: -2.56%

RMSE Change: -9.18%

MAPE Change: -38.43%

# Baseline Model With Lagged Target Features

- Lagged features are past values of a time series used to capture autocorrelation (i.e. the relationship between a data point and its previous values). 

- This is based on the idea that the past influences the future.

- By including lagged features, the model gains a historical context that helps it identify:

  - `Trends`: Long-term upward or downward movements.

  - `Seasonality`: Regular, repeating patterns, such as daily or weekly cycles.

  - `Temporal dependencies`: How a value at one time step is directly related to its value at a previous time step.

In [21]:
import narwhals as nw


def _calculate_corr(x: list[float] | np.ndarray, y: list[float] | np.ndarray) -> float:
    return np.corrcoef(x, y)[0][1].item()


def compute_autocorrelation(series: nw.Series, max_lag: int = 24) -> dict[int, float]:
    """
    Compute autocorrelation for a time series using Polars.

    Parameters
    ----------
    series : pl.Series
        The time series data.
    max_lag : int, default=24
        Maximum lag to compute autocorrelation for.

    Returns
    -------
    dict[int, float]
        Dictionary mapping lag to autocorrelation value.
    """
    autocorr_values: dict[int, float] = {}

    for lag in range(1, max_lag + 1):
        try:
            # Create a DataFrame with original and lagged series
            df_corr: nw.DataFrame = nw.from_native(
                pl.DataFrame({"original": series, "lagged": series.shift(lag)})
            ).drop_nulls()

            # Compute correlation if we have sufficient data
            if df_corr.shape[0] > 1:
                correlation = _calculate_corr(df_corr["original"], df_corr["lagged"])
                autocorr_values[lag] = correlation
            else:
                autocorr_values[lag] = None

        except Exception as e:
            console.print(f"Error computing lag {lag}: {e}", style="error")
            autocorr_values[lag] = None

    return autocorr_values

In [22]:
# Compute autocorrelation of the target variable
auto_correlation: dict[int, float] = compute_autocorrelation(
    series=df["target"], max_lag=24
)
for lag, autocorr in auto_correlation.items():
    console.print(f"Lag {lag}: Autocorrelation = {autocorr:.3f}", style="info")

Lag 1: Autocorrelation = 0.845

Lag 2: Autocorrelation = 0.597

Lag 3: Autocorrelation = 0.407

Lag 4: Autocorrelation = 0.274

Lag 5: Autocorrelation = 0.146

...

Lag 20: Autocorrelation = 0.262

Lag 21: Autocorrelation = 0.371

Lag 22: Autocorrelation = 0.520

Lag 23: Autocorrelation = 0.705

Lag 24: Autocorrelation = 0.809

### Comment

- **Strong Hourly Dependency**: The high correlation at Lag 1 (r=0.845) shows that bike rentals are highly dependent on the previous hour's count.

- **Clear Daily Seasonality**: The strong correlation at Lag 24 (r=0.809) confirms a predictable daily pattern in bike usage.

- **Inverse Peak/Off-Peak Relationship**: The negative correlations between 6 and 18 hours apart indicate that peak and off-peak periods are inversely related.

- **Modeling Strategy**: Lagged features (especially 1, 2, 23, and 24 hours) and time-of-day features will be essential for building an effective model.

In [23]:
columns_to_drop: list[str] = ["yr", "atemp", "casual", "registered", "datetime"]
df: pl.DataFrame = data.clone().drop(columns_to_drop)

df.head(3)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64
1,1,0,0,6,0,1,0.24,0.81,0.0,16
1,1,1,0,6,0,1,0.22,0.8,0.0,40
1,1,2,0,6,0,1,0.22,0.8,0.0,32


In [24]:
# Shift the `cnt` column by one hour to create the target variable
# i.e. predict the next hour's demand based on current and past data
df = df.with_columns(pl.col("cnt").shift(-1).alias("target")).with_columns(
    pl.col("target").fill_null(strategy="forward")
)
display(df.null_count())
df.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0


season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1


In [25]:
# Add lags 1, 23 and 24 hours
df = df.with_columns(
    [
        pl.col("cnt").alias("current"),
        pl.col("cnt").shift(1).alias("cnt_lag_1"),
        pl.col("cnt").shift(2).alias("cnt_lag_2"),
        pl.col("cnt").shift(23).alias("cnt_lag_23"),
        pl.col("cnt").shift(24).alias("cnt_lag_24"),
    ]
).fill_null(strategy="backward")
print("Check for null values after adding lag features:")
display(df.null_count())
df.head()

Check for null values after adding lag features:


season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,current,cnt_lag_1,cnt_lag_2,cnt_lag_23,cnt_lag_24
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,current,cnt_lag_1,cnt_lag_2,cnt_lag_23,cnt_lag_24
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40,16,16,16,16,16
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32,40,16,16,16,16
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13,32,40,16,16,16
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1,13,32,40,16,16
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1,1,13,32,16,16


In [26]:
df = df.drop(["cnt"])

train_df, test_df = split_temporal_data(df)

x_train = train_df.drop("target").to_numpy()
y_train = train_df["target"].to_numpy()

x_test = test_df.drop("target").to_numpy()
y_test = test_df["target"].to_numpy()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((11122, 15), (11122,), (2781, 15), (2781,))

In [27]:
rf_reg = RandomForestRegressor(random_state=123)
rf_reg.fit(x_train, y_train)
rf_reg.score(x_test, y_test)

y_pred = rf_reg.predict(x_test)

In [28]:
metrics_base_with_lagged_feats = compute_metrics(y_test, y_pred)
metrics_base_with_lagged_feats

{'MAE': 43.48, 'RMSE': 71.07, 'MAPE': 24.61}

In [29]:
plot_actual_vs_predicted(
    y_true=y_test,
    y_pred=y_pred,
    n=n,
    title="Actual vs Predicted Bike Rental Counts (Base Model with Lagged Features)",
)

Using first 300 values for plotting.


In [30]:
console.print(f"Metrics (Base Model): {metrics_base}", style="info")
console.print(f"Metrics (Dummy Model): {metrics_dummy}", style="info")
console.print(
    f"Metrics (Base Model with Lagged Features): {metrics_base_with_lagged_feats}",
    style="info",
)

Metrics (Base Model): {'MAE': 84.78, 'RMSE': 116.41, 'MAPE': 34.11}

Metrics (Dummy Model): {'MAE': 87.01, 'RMSE': 128.17, 'MAPE': 55.4}

Metrics (Base Model with Lagged Features): {'MAE': 43.48, 'RMSE': 71.07, 'MAPE': 24.61}

In [31]:
mae_base, mae_dummy = (
    metrics_base_with_lagged_feats.get("MAE"),
    metrics_dummy.get("MAE"),
)
rmse_base, rmse_dummy = (
    metrics_base_with_lagged_feats.get("RMSE"),
    metrics_dummy.get("RMSE"),
)
mape_base, mape_dummy = (
    metrics_base_with_lagged_feats.get("MAPE"),
    metrics_dummy.get("MAPE"),
)

console.print(
    f"MAE Change: {pct_change(old=mae_dummy, new=mae_base):.2f}%", style="highlight"
)
console.print(
    f"RMSE Change: {pct_change(old=rmse_dummy, new=rmse_base):.2f}%", style="highlight"
)
console.print(
    f"MAPE Change: {pct_change(old=mape_dummy, new=mape_base):.2f}%", style="highlight"
)

MAE Change: -50.03%

RMSE Change: -44.55%

MAPE Change: -55.58%

In [32]:
mae_new, mae_old = (
    metrics_base_with_lagged_feats.get("MAE"),
    metrics_base.get("MAE"),
)
rmse_new, rmse_old = (
    metrics_base_with_lagged_feats.get("RMSE"),
    metrics_base.get("RMSE"),
)
mape_new, mape_old = (
    metrics_base_with_lagged_feats.get("MAPE"),
    metrics_base.get("MAPE"),
)

console.print(
    f"MAE Change: {pct_change(old=mae_old, new=mae_new):.2f}%", style="highlight"
)
console.print(
    f"RMSE Change: {pct_change(old=rmse_old, new=rmse_new):.2f}%", style="highlight"
)
console.print(
    f"MAPE Change: {pct_change(old=mape_old, new=mape_new):.2f}%", style="highlight"
)

MAE Change: -48.71%

RMSE Change: -38.95%

MAPE Change: -27.85%

# Add More Features

- Add `temporal` features
  - `dayofweek`
  - `month`
  - `is_weekend`
  - `is_holiday`

- Add `seasonal` features
  - spring, summer, fall, winter
  - `sin_hour`, `cos_hour`, `sin_day_of_week`, `cos_day_of_week`

- Add `trend` features
  - `cnt_diff_1hr`, `cnt_diff_3hr`, `cnt_diff_6hr`, `cnt_diff_24hr`

- Add `lagged` features
  - `cnt_lag_1`, `cnt_lag_2`, `cnt_lag_23`, `cnt_lag_24`
  - `hr_lag_1`, `hr_lag_2`

- Add `exogenous` (not time-based) features
  - `temp`, `hum`, `windspeed`, `weather`
  - `holiday`, `event`
  - `location` (if available)

- Add `derived` features
  - `temp_change_1hr`, `temp_change_3hr`
  - Add `rolling` features
    - `cnt_roll_mean_3hr`, `cnt_roll_std_3hr`
    - `cnt_roll_mean_6hr`, `cnt_roll_std_6hr`
    - `cnt_roll_mean_24hr`, `cnt_roll_std_24hr`
  - Add `interaction` features
    - `temp_hum_interaction`, `temp_wind_interaction`
  - Add `binary` features
    - `is_high_temp`, `is_low_hum`, `is_peak_hour`, `is_working_hour`, `is_business_hour`, `is_weekend`

In [33]:
columns_to_drop: list[str] = ["yr", "atemp", "casual", "registered", "datetime"]
df: pl.DataFrame = data.clone().drop(columns_to_drop)

# Shift the `cnt` column by one hour to create the target variable
# i.e. predict the next hour's demand based on current and past data
df = df.with_columns(pl.col("cnt").shift(-1).alias("target")).with_columns(
    pl.col("target").fill_null(strategy="forward")
)
df.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1


### Add More Temporal Features

In [34]:
df = df.with_columns(
    (pl.col("weekday").is_in([5, 6])).cast(pl.Int8).alias("is_weekend")
)

df.head(2)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,is_weekend
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i8
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40,1
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32,1


### Add More Seasonal Features

- Add `sin` and `cos` transformations for cyclical features like hour of the day and day of the week.

$$sin\_hour = sin(val \times \frac{2\pi}{24})$$
$$cos\_hour = cos(val \times \frac{2\pi}{24})$$
$$sin\_weekday = sin(val \times \frac{2\pi}{7})$$
$$cos\_weekday = cos(val \times \frac{2\pi}{7})$$

In [35]:
df = df.with_columns(
    (pl.col("hr") * (2 * np.pi / 24)).sin().alias("sin_hour"),
    (pl.col("hr") * (2 * np.pi / 24)).cos().alias("cos_hour"),
    (pl.col("weekday") * (2 * np.pi / 7)).sin().alias("sin_weekday"),
    (pl.col("weekday") * (2 * np.pi / 7)).cos().alias("cos_weekday"),
)

df.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i8,f64,f64,f64,f64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40,1,0.0,1.0,-0.781831,0.62349
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32,1,0.258819,0.965926,-0.781831,0.62349
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13,1,0.5,0.866025,-0.781831,0.62349
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1,1,0.707107,0.707107,-0.781831,0.62349
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1,1,0.866025,0.5,-0.781831,0.62349


In [36]:
n: int = 100

fig = go.Figure()

# Add actual values trace
fig.add_trace(
    go.Scatter(
        x=list(range(n)),
        y=df["sin_hour"][:n],
        mode="lines",
        name="Hour (sin)",
        line={"color": "blue", "width": 2},
    )
)

# Add predicted values trace
fig.add_trace(
    go.Scatter(
        x=list(range(n)),
        y=df["cos_hour"][:n],
        mode="lines",
        name="Hour (cos)",
        line={"color": "red", "width": 2},
    )
)

# Update layout for better visualization
fig.update_layout(
    title="</b>Cyclical Features: Sine and Cosine Transformations of Hour of Day</b>",
    xaxis_title="Time Steps",
    yaxis_title="Transformed Value",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500,
)

# Show the interactive plot
fig.show()

In [ ]:
auto_correlation: dict[int, float] = compute_autocorrelation(
    series=df["temp"], max_lag=24
)

sorted_auto_correlation = sorted(
    auto_correlation.items(),
    key=lambda item: item[1] if item[1] is not None else -1,
    reverse=True,
)
sorted_auto_correlation[:10]

[(1, 0.9925069855541919),
 (2, 0.981283920572705),
 (3, 0.965779929192312),
 (4, 0.9472661046674967),
 (5, 0.9271454013371131),
 (23, 0.9143308473785758),
 (24, 0.9134493764203966),
 (22, 0.9104627601429008),
 (6, 0.9067865271191217),
 (21, 0.9025653925620295)]

In [ ]:
auto_correlation: dict[int, float] = compute_autocorrelation(
    series=df["hum"], max_lag=24
)

sorted_auto_correlation = sorted(
    auto_correlation.items(),
    key=lambda item: item[1] if item[1] is not None else -1,
    reverse=True,
)
sorted_auto_correlation[:10]

[(1, 0.95011035078593),
 (2, 0.8913004002293619),
 (3, 0.8192387808004927),
 (4, 0.7376664285470248),
 (5, 0.6520438180037119),
 (6, 0.567206978881164),
 (23, 0.4911800511124771),
 (24, 0.49054695336399995),
 (7, 0.4853189164438995),
 (22, 0.476470193927108)]

In [ ]:
auto_correlation: dict[int, float] = compute_autocorrelation(
    series=df["hr"], max_lag=24
)

sorted_auto_correlation = sorted(
    auto_correlation.items(),
    key=lambda item: item[1] if item[1] is not None else -1,
    reverse=True,
)
sorted_auto_correlation[:10]

[(24, 0.9640175659452095),
 (23, 0.7765710630489999),
 (1, 0.7584328002756796),
 (22, 0.5597192084615917),
 (2, 0.5377623840211717),
 (21, 0.35868341735813386),
 (3, 0.33836951566119616),
 (20, 0.17897499224973265),
 (4, 0.1609548142422359),
 (19, 0.021014626959614716)]

### Add Trend Features

- Add `diff`/`change` transformations to capture trends over different time horizons.
- e.g. `cnt_diff_1hr`, `cnt_diff_24hr`, `temp_diff_1hr`, `temp_diff_3hr`, `hum_diff_1hr`, `hum_diff_2hr`

In [40]:
df = df.with_columns(
    pl.col("cnt").diff(1).alias("cnt_diff_1hr"),
    pl.col("cnt").diff(24).alias("cnt_diff_24hr"),
    pl.col("hr").diff(1).alias("hr_diff_1hr"),
    pl.col("hr").diff(23).alias("hr_diff_23hr"),
    pl.col("hr").diff(24).alias("hr_diff_24hr"),
    pl.col("temp").diff(1).alias("temp_diff_1hr"),
    pl.col("temp").diff(3).alias("temp_diff_3hr"),
    pl.col("temp").diff(24).alias("temp_diff_24hr"),
    pl.col("hum").diff(1).alias("hum_diff_1hr"),
    pl.col("hum").diff(3).alias("hum_diff_3hr"),
).fill_null(strategy="backward")

df.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_diff_1hr,cnt_diff_24hr,hr_diff_1hr,hr_diff_23hr,hr_diff_24hr,temp_diff_1hr,temp_diff_3hr,temp_diff_24hr,hum_diff_1hr,hum_diff_3hr
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40,1,0.0,1.0,-0.781831,0.62349,24,1,1,23,0,-0.02,0.0,0.22,-0.01,-0.06
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32,1,0.258819,0.965926,-0.781831,0.62349,24,1,1,23,0,-0.02,0.0,0.22,-0.01,-0.06
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13,1,0.5,0.866025,-0.781831,0.62349,-8,1,1,23,0,0.0,0.0,0.22,0.0,-0.06
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1,1,0.707107,0.707107,-0.781831,0.62349,-19,1,1,23,0,0.02,0.0,0.22,-0.05,-0.06
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1,1,0.866025,0.5,-0.781831,0.62349,-12,1,1,23,0,0.0,0.02,0.22,0.0,-0.05


### Add Lagged Features

- Add lagged features for the target variable and other relevant features based on autocorrelation analysis.
- e.g. `cnt_lag_1`, `cnt_lag_2`, `cnt_lag_23`, `cnt_lag_24`, `temp_lag_1`, `temp_lag_2`, `hum_lag_1`, `hum_lag_2`

In [41]:
# Add lags
df = df.with_columns(
    [
        pl.col("cnt").alias("current"),  # current value
        pl.col("cnt").shift(1).alias("cnt_lag_1"),
        pl.col("cnt").shift(2).alias("cnt_lag_2"),
        pl.col("cnt").shift(23).alias("cnt_lag_23"),
        pl.col("cnt").shift(24).alias("cnt_lag_24"),
        pl.col("hr").shift(1).alias("hr_lag_1"),
        pl.col("hr").shift(23).alias("hr_lag_23"),
        pl.col("hr").shift(24).alias("hr_lag_24"),
        pl.col("temp").shift(1).alias("temp_lag_1"),
        pl.col("temp").shift(2).alias("temp_lag_2"),
        pl.col("temp").shift(3).alias("temp_lag_3"),
        pl.col("hum").shift(1).alias("hum_lag_1"),
        pl.col("hum").shift(2).alias("hum_lag_2"),
        pl.col("hum").shift(3).alias("hum_lag_3"),
    ]
).fill_null(strategy="backward")

df.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_diff_1hr,cnt_diff_24hr,hr_diff_1hr,hr_diff_23hr,hr_diff_24hr,temp_diff_1hr,temp_diff_3hr,temp_diff_24hr,hum_diff_1hr,hum_diff_3hr,current,cnt_lag_1,cnt_lag_2,cnt_lag_23,cnt_lag_24,hr_lag_1,hr_lag_23,hr_lag_24,temp_lag_1,temp_lag_2,temp_lag_3,hum_lag_1,hum_lag_2,hum_lag_3
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40,1,0.0,1.0,-0.781831,0.62349,24,1,1,23,0,-0.02,0.0,0.22,-0.01,-0.06,16,16,16,16,16,0,0,0,0.24,0.24,0.24,0.81,0.81,0.81
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32,1,0.258819,0.965926,-0.781831,0.62349,24,1,1,23,0,-0.02,0.0,0.22,-0.01,-0.06,40,16,16,16,16,0,0,0,0.24,0.24,0.24,0.81,0.81,0.81
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13,1,0.5,0.866025,-0.781831,0.62349,-8,1,1,23,0,0.0,0.0,0.22,0.0,-0.06,32,40,16,16,16,1,0,0,0.22,0.24,0.24,0.8,0.81,0.81
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1,1,0.707107,0.707107,-0.781831,0.62349,-19,1,1,23,0,0.02,0.0,0.22,-0.05,-0.06,13,32,40,16,16,2,0,0,0.22,0.22,0.24,0.8,0.8,0.81
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1,1,0.866025,0.5,-0.781831,0.62349,-12,1,1,23,0,0.0,0.02,0.22,0.0,-0.05,1,13,32,16,16,3,0,0,0.24,0.22,0.22,0.75,0.8,0.8


### Add Exogenous Features

- The data already includes some exogenous features like `temp`, `hum`, `windspeed`, `holidays`.
- We don't have additional exogenous features (e.g., events, promotions, etc.) in this dataset, but if we did, we could consider adding them here.

<br>

### Add Derived Features

- Add `derived` features based on domain knowledge and exploratory data analysis.
- e.g. `temp_hum_interaction` (interaction between temperature and humidity), `temp_hr_interaction` (interaction between temperature and hour of the day), `hum_hr_interaction` (interaction between humidity and hour of the day), etc.
- Add `rolling` features to capture short-term trends and variability.
- e.g. `cnt_roll_mean_3hr`, `cnt_roll_std_3hr`, `cnt_roll_mean_6hr`, `cnt_roll_std_6hr`, `cnt_roll_mean_24hr`, `cnt_roll_std_24hr`, etc.
- Add `binary` features to capture specific conditions.
- e.g. `is_high_temp` (1 if temp > 25C else 0), `is_low_hum` (1 if hum < 30% else 0), `is_peak_hour` (1 if hr in [7-9, 16-18] else 0), `is_working_hour` (1 if hr in [9-17] else 0), `is_business_hour` (1 if hr in [8-20] else 0), `is_weekend` (1 if dayofweek in [5, 6] else 0), etc.
- Add features based on interactions between different variables..

In [42]:
display(df["temp"].describe())
np.percentile(df["temp"], 80).item()

statistic,value
str,f64
"""count""",13903.0
"""null_count""",0.0
"""mean""",0.49917
"""std""",0.197716
"""min""",0.02
"""25%""",0.34
"""50%""",0.5
"""75%""",0.66
"""max""",1.0


0.7

In [43]:
# temp_hum_interaction = temperature * humidity
# temp_hr_interaction = temperature * hour of the day
# hum_hr_interaction = humidity * hour of the day
# rolling_mean_temp_3hr = rolling mean of temperature over the past 3 hours
# rolling_mean_hum_3hr = rolling mean of humidity over the past 3 hours
# is_high_temp = 1 if temperature > 25 else 0
# is_high_hum = 1 if humidity > 75 else 0
# is_peak_hour = 1 if hr in [7-9, 16-18] else 0
# is_working_hour = 1 if hr in [9-17] else 0
# is_business_hour = 1 if hr in [8-20] else 0
high_percentile_temp: float = np.percentile(df["temp"], 80).item()
high_percentile_hum: float = np.percentile(df["hum"], 80).item()
working_hours: list[int] = [9, 10, 11, 12, 13, 14, 15, 16, 17]
business_hours: list[int] = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
peak_hours: list[int] = [7, 8, 9, 16, 17, 18]

df = df.with_columns(
    [
        # Interaction features
        (pl.col("temp") * pl.col("hum")).alias("temp_hum_interaction"),
        (pl.col("temp") * pl.col("hr")).alias("temp_hr_interaction"),
        (pl.col("hum") * pl.col("hr")).alias("hum_hr_interaction"),
        pl.col("temp").rolling_mean(window_size=3).alias("rolling_mean_temp_3hr"),
        pl.col("temp").rolling_median(window_size=3).alias("rolling_median_temp_3hr"),
        pl.col("hum").rolling_mean(window_size=3).alias("rolling_mean_hum_3hr"),
        pl.col("hum").rolling_median(window_size=3).alias("rolling_median_hum_3hr"),
        # Binary features
        (pl.col("temp") > high_percentile_temp).cast(pl.Int8).alias("is_high_temp"),
        (pl.col("hum") > high_percentile_hum).cast(pl.Int8).alias("is_high_hum"),
        (pl.col("hr").is_in(peak_hours)).cast(pl.Int8).alias("is_peak_hour"),
        (pl.col("hr").is_in(working_hours)).cast(pl.Int8).alias("is_working_hour"),
        (pl.col("hr").is_in(business_hours)).cast(pl.Int8).alias("is_business_hour"),
    ]
).fill_null(strategy="backward")
print("Check for null values after adding lag features:")
display(df.null_count().sum_horizontal())

df.head()

Check for null values after adding lag features:


sum
u32
0


season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,target,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_diff_1hr,cnt_diff_24hr,hr_diff_1hr,hr_diff_23hr,hr_diff_24hr,temp_diff_1hr,temp_diff_3hr,temp_diff_24hr,hum_diff_1hr,hum_diff_3hr,current,cnt_lag_1,cnt_lag_2,cnt_lag_23,cnt_lag_24,hr_lag_1,hr_lag_23,hr_lag_24,temp_lag_1,temp_lag_2,temp_lag_3,hum_lag_1,hum_lag_2,hum_lag_3,temp_hum_interaction,temp_hr_interaction,hum_hr_interaction,rolling_mean_temp_3hr,rolling_median_temp_3hr,rolling_mean_hum_3hr,rolling_median_hum_3hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8
1,1,0,0,6,0,1,0.24,0.81,0.0,16,40,1,0.0,1.0,-0.781831,0.62349,24,1,1,23,0,-0.02,0.0,0.22,-0.01,-0.06,16,16,16,16,16,0,0,0,0.24,0.24,0.24,0.81,0.81,0.81,0.1944,0.0,0.0,0.226667,0.22,0.803333,0.8,0,0,0,0,0
1,1,1,0,6,0,1,0.22,0.8,0.0,40,32,1,0.258819,0.965926,-0.781831,0.62349,24,1,1,23,0,-0.02,0.0,0.22,-0.01,-0.06,40,16,16,16,16,0,0,0,0.24,0.24,0.24,0.81,0.81,0.81,0.176,0.22,0.8,0.226667,0.22,0.803333,0.8,0,0,0,0,0
1,1,2,0,6,0,1,0.22,0.8,0.0,32,13,1,0.5,0.866025,-0.781831,0.62349,-8,1,1,23,0,0.0,0.0,0.22,0.0,-0.06,32,40,16,16,16,1,0,0,0.22,0.24,0.24,0.8,0.81,0.81,0.176,0.44,1.6,0.226667,0.22,0.803333,0.8,0,0,0,0,0
1,1,3,0,6,0,1,0.24,0.75,0.0,13,1,1,0.707107,0.707107,-0.781831,0.62349,-19,1,1,23,0,0.02,0.0,0.22,-0.05,-0.06,13,32,40,16,16,2,0,0,0.22,0.22,0.24,0.8,0.8,0.81,0.18,0.72,2.25,0.226667,0.22,0.783333,0.8,0,0,0,0,0
1,1,4,0,6,0,1,0.24,0.75,0.0,1,1,1,0.866025,0.5,-0.781831,0.62349,-12,1,1,23,0,0.0,0.02,0.22,0.0,-0.05,1,13,32,16,16,3,0,0,0.24,0.22,0.22,0.75,0.8,0.8,0.18,0.96,3.0,0.233333,0.24,0.766667,0.75,0,0,0,0,0


In [44]:
df = df.drop(["cnt"])

train_df, test_df = split_temporal_data(df)

x_train = train_df.drop("target").to_numpy()
y_train = train_df["target"].to_numpy()

x_test = test_df.drop("target").to_numpy()
y_test = test_df["target"].to_numpy()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((11122, 51), (11122,), (2781, 51), (2781,))

In [45]:
rf_reg = RandomForestRegressor(random_state=123)
rf_reg.fit(x_train, y_train)
rf_reg.score(x_test, y_test)

y_pred = rf_reg.predict(x_test)

In [46]:
metrics_base_with_advanced_feats = compute_metrics(y_test, y_pred)
metrics_base_with_advanced_feats

{'MAE': 41.65, 'RMSE': 65.37, 'MAPE': 24.58}

In [47]:
mae_new, mae_old = (
    metrics_base_with_advanced_feats.get("MAE"),
    metrics_base.get("MAE"),
)
rmse_new, rmse_old = (
    metrics_base_with_advanced_feats.get("RMSE"),
    metrics_base.get("RMSE"),
)
mape_new, mape_old = (
    metrics_base_with_advanced_feats.get("MAPE"),
    metrics_base.get("MAPE"),
)

console.print(
    f"MAE Change: {pct_change(old=mae_old, new=mae_new):.2f}%", style="highlight"
)
console.print(
    f"RMSE Change: {pct_change(old=rmse_old, new=rmse_new):.2f}%", style="highlight"
)
console.print(
    f"MAPE Change: {pct_change(old=mape_old, new=mape_new):.2f}%", style="highlight"
)

MAE Change: -50.87%

RMSE Change: -43.85%

MAPE Change: -27.94%

In [48]:
plot_actual_vs_predicted(
    y_true=y_test,
    y_pred=y_pred,
    n=n,
    title="Actual vs Predicted Bike Rental Counts (Model with Advanced Features)",
)

Using first 100 values for plotting.


In [49]:
console.print(f"Metrics (Base Model): {metrics_base}", style="info")
console.print(f"Metrics (Dummy Model): {metrics_dummy}", style="info")
console.print(
    f"Metrics (Base Model with Lagged Features): {metrics_base_with_lagged_feats}",
    style="info",
)
console.print(
    f"Metrics (Base Model with Advanced Features): {metrics_base_with_advanced_feats}",
    style="info",
)

Metrics (Base Model): {'MAE': 84.78, 'RMSE': 116.41, 'MAPE': 34.11}

Metrics (Dummy Model): {'MAE': 87.01, 'RMSE': 128.17, 'MAPE': 55.4}

Metrics (Base Model with Lagged Features): {'MAE': 43.48, 'RMSE': 71.07, 'MAPE': 24.61}

Metrics (Base Model with Advanced Features): {'MAE': 41.65, 'RMSE': 65.37, 'MAPE': 24.58}

<br>

# Train With TimeSeriesSplit

In [50]:
from sklearn.model_selection import TimeSeriesSplit

In [51]:
# df = df.drop(["cnt"])

train_df, test_df = split_temporal_data(df)

x_train = train_df.drop("target").to_numpy()
y_train = train_df["target"].to_numpy()

x_test = test_df.drop("target").to_numpy()
y_test = test_df["target"].to_numpy()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(11122, 51) (11122,) (2781, 51) (2781,)


In [52]:
n_splits: int = 5
test_size: int = 168  # 1 week of hourly data

tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size, gap=0)
rf_reg = RandomForestRegressor(random_state=123)
all_rmse: list[float] = []
all_mae: list[float] = []
all_mape: list[float] = []
print(tscv)

for i, (train_index, test_index) in enumerate(tscv.split(x_train), start=1):
    print(f"Fold {i}:")
    x_tr, x_val = x_train[train_index], x_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    # Train the model
    rf_reg.fit(x_tr, y_tr)
    # Evaluate the model
    y_pred = rf_reg.predict(x_val)
    metrics = compute_metrics(y_val, y_pred)
    print(f"Validation Metrics: {metrics}")
    all_rmse.append(metrics.get("RMSE"))
    all_mae.append(metrics.get("MAE"))
    all_mape.append(metrics.get("MAPE"))

print("\nCross-Validation Results:")
print(f"Average MAE over {n_splits} folds: {np.mean(all_mae).round(2)}")
print(f"Average RMSE over {n_splits} folds: {np.mean(all_rmse).round(2)}")
print(f"Average MAPE over {n_splits} folds: {np.mean(all_mape).round(2)}")

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=168)
Fold 1:
Validation Metrics: {'MAE': 36.47, 'RMSE': 65.54, 'MAPE': 29.43}
Fold 2:
Validation Metrics: {'MAE': 53.59, 'RMSE': 83.45, 'MAPE': 25.27}
Fold 3:
Validation Metrics: {'MAE': 36.69, 'RMSE': 62.04, 'MAPE': 30.1}
Fold 4:
Validation Metrics: {'MAE': 36.13, 'RMSE': 58.41, 'MAPE': 25.88}
Fold 5:
Validation Metrics: {'MAE': 36.85, 'RMSE': 59.81, 'MAPE': 27.68}

Cross-Validation Results:
Average MAE over 5 folds: 39.95
Average RMSE over 5 folds: 65.85
Average MAPE over 5 folds: 27.67


In [53]:
# Evaluate on the test set
y_test_pred = rf_reg.predict(x_test)
metrics_test = compute_metrics(y_test, y_test_pred)
print(f"\nTest Set Metrics: {metrics_test}")


Test Set Metrics: {'MAE': 42.23, 'RMSE': 66.45, 'MAPE': 24.84}


# Tune Hyperparameters Using Optuna

In [54]:
import optuna
from optuna.trial import TrialState

In [55]:
# Step 1: Define the Optuna objective with TimeSeriesSplit on x_train
def objective(trial: optuna.Trial) -> float:
    """Objective function for Optuna hyperparameter optimization using TimeSeriesSplit."""
    np.random.seed(42)  # noqa: NPY002
    params: dict[str, Any] = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical(
            "max_features", ["sqrt", "log2", None]
        ),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42,
    }

    # Use TimeSeriesSplit for cross-validation
    n_splits: int = 5
    test_size: int = 168  # 1 week of hourly data
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size, gap=0)

    cv_scores: list[float] = []

    for train_idx, val_idx in tscv.split(x_train):
        x_tr, x_val = x_train[train_idx], x_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model = RandomForestRegressor(**params)
        model.fit(x_tr, y_tr)

        preds = model.predict(x_val)
        rmse = root_mean_squared_error(y_val, preds)
        cv_scores.append(rmse)

    # Return mean RMSE across all CV folds
    mean_rmse = np.mean(cv_scores)

    # Store additional metrics for analysis
    trial.set_user_attr("cv_scores", cv_scores)
    trial.set_user_attr("cv_std", np.std(cv_scores))

    return mean_rmse

In [56]:
# Step 2: Run the Optuna study

n_trials = 50
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=42),
)
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

best_trial: optuna.Trial = study.best_trial

best_params: dict[str, Any] = best_trial.params

[I 2025-09-29 13:23:36,925] A new study created in memory with name: no-name-4e8b8466-e09a-4e95-bb4f-f24956cf371d


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-09-29 13:23:47,832] Trial 0 finished with value: 75.67012526653788 and parameters: {'n_estimators': 144, 'max_depth': 20, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 75.67012526653788.
[I 2025-09-29 13:23:54,171] Trial 1 finished with value: 132.32503729813095 and parameters: {'n_estimators': 227, 'max_depth': 3, 'min_samples_split': 20, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 75.67012526653788.
[I 2025-09-29 13:24:46,007] Trial 2 finished with value: 67.94649944285518 and parameters: {'n_estimators': 158, 'max_depth': 8, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': True}. Best is trial 2 with value: 67.94649944285518.
[I 2025-09-29 13:26:06,604] Trial 3 finished with value: 66.67990887410261 and parameters: {'n_estimators': 179, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 7, 'max_features': No

In [57]:
# Display optimization results
best_rmse: float = best_trial.value

console.print("=" * 50, style="highlight")
console.print("HYPERPARAMETER OPTIMIZATION RESULTS", style="success")
console.print("=" * 50, style="highlight")

console.print(f"Best RMSE: {best_rmse:.4f}", style="success")
console.print(f"Best parameters: {best_params}", style="info")

==================================================

HYPERPARAMETER OPTIMIZATION RESULTS

==================================================

Best RMSE: 64.4834

Best parameters: {'n_estimators': 149, 'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 2, 
'max_features': None, 'bootstrap': True}

In [58]:
# For debugging purposes
console.print(best_trial.user_attrs)
console.print(study.trials[:2])

{
    'cv_scores': [60.552316381414144, 84.26042571734928, 60.74327509620594, 56.47043301651862, 60.39061800566645],
    'cv_std': np.float64(10.015300327674275)
}

[
    FrozenTrial(number=0, state=1, values=[75.67012526653788], datetime_start=datetime.datetime(2025, 9, 29, 13, 
23, 36, 960790), datetime_complete=datetime.datetime(2025, 9, 29, 13, 23, 47, 832002), params={'n_estimators': 144,
'max_depth': 20, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': True}, 
user_attrs={'cv_scores': [65.61124755517089, 106.17138708398257, 76.8762204633419, 72.6632649775868, 
57.0285062526072], 'cv_std': np.float64(16.670415874537213)}, system_attrs={}, intermediate_values={}, 
distributions={'n_estimators': IntDistribution(high=300, log=False, low=50, step=1), 'max_depth': 
IntDistribution(high=20, log=False, low=3, step=1), 'min_samples_split': IntDistribution(high=20, log=False, low=2,
step=1), 'min_samples_leaf': IntDistribution(high=10, log=False, low=1, step=1), 'max_features': 
CategoricalDistribution(choices=('sqrt', 'log2', None)), 'bootstrap': CategoricalDistribution(choices=(True, 
False))}, trial_id=0, value=None),
    FrozenTrial(number=1, state=1, values=[132.32503729813095], datetime_start=datetime.datetime(2025, 9, 29, 13, 
23, 47, 837069), datetime_complete=datetime.datetime(2025, 9, 29, 13, 23, 54, 171558), params={'n_estimators': 227,
'max_depth': 3, 'min_samples_split': 20, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': False}, 
user_attrs={'cv_scores': [116.52755128714304, 156.47700510865215, 129.47912742872316, 138.04303597107534, 
121.09846669506099], 'cv_std': np.float64(14.140075761480997)}, system_attrs={}, intermediate_values={}, 
distributions={'n_estimators': IntDistribution(high=300, log=False, low=50, step=1), 'max_depth': 
IntDistribution(high=20, log=False, low=3, step=1), 'min_samples_split': IntDistribution(high=20, log=False, low=2,
step=1), 'min_samples_leaf': IntDistribution(high=10, log=False, low=1, step=1), 'max_features': 
CategoricalDistribution(choices=('sqrt', 'log2', None)), 'bootstrap': CategoricalDistribution(choices=(True, 
False))}, trial_id=1, value=None)
]

In [59]:
# Display cross-validation statistics
cv_scores: list[float] = best_trial.user_attrs.get("cv_scores", [])
cv_std: float = best_trial.user_attrs.get("cv_std", 0.0)

console.print(f"CV Scores: {np.mean(cv_scores):.4f}", style="info")
console.print(f"CV Standard Deviation: {cv_std:.4f}", style="info")

# Study statistics
total_trials: int = len(study.trials)
failed_trials: int = len([t for t in study.trials if t.state == TrialState.FAIL])
complete_trials: int = len([t for t in study.trials if t.state == TrialState.COMPLETE])
pruned_trials: int = len([t for t in study.trials if t.state == TrialState.PRUNED])

console.print(f"Total trials: {total_trials}", style="info")
console.print(f"Complete trials: {complete_trials}", style="success")
console.print(
    f"Pruned trials: {pruned_trials}", style="warning" if pruned_trials > 0 else "info"
)
console.print(
    f"Failed trials: {failed_trials}", style="error" if failed_trials > 0 else "info"
)

# Display trial state distribution
console.print(
    f"Success rate: {(complete_trials / total_trials) * 100:.1f}%", style="success"
)

CV Scores: 64.4834

CV Standard Deviation: 10.0153

Total trials: 50

Complete trials: 50

Pruned trials: 0

Failed trials: 0

Success rate: 100.0%

In [60]:
# Step 4: Train final model on full x_train with best parameters
console.print("\nTraining final model with optimized parameters...", style="info")

final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(x_train, y_train)

# Evaluate on test set
y_test_pred_optimized: np.ndarray = final_rf_model.predict(x_test)
final_metrics_optimized: dict[str, float] = compute_metrics(
    y_test, y_test_pred_optimized
)

console.print("=" * 40, style="highlight")
console.print("FINAL MODEL EVALUATION", style="success")
console.print("=" * 40, style="highlight")

console.print(f"Optimized Test Metrics: {final_metrics_optimized}", style="success")

# Compare with previous best model
if "metrics_base_with_advanced_feats" in locals():
    mae_improvement: float = pct_change(
        old=metrics_base_with_advanced_feats.get("MAE", 0),
        new=final_metrics_optimized.get("MAE", 0),
    )
    rmse_improvement: float = pct_change(
        old=metrics_base_with_advanced_feats.get("RMSE", 0),
        new=final_metrics_optimized.get("RMSE", 0),
    )
    mape_improvement: float = pct_change(
        old=metrics_base_with_advanced_feats.get("MAPE", 0),
        new=final_metrics_optimized.get("MAPE", 0),
    )

    console.print(f"MAE Improvement: {mae_improvement:.2f}%", style="highlight")
    console.print(f"RMSE Improvement: {rmse_improvement:.2f}%", style="highlight")
    console.print(f"MAPE Improvement: {mape_improvement:.2f}%", style="highlight")

Training final model with optimized parameters...

========================================

FINAL MODEL EVALUATION

========================================

Optimized Test Metrics: {'MAE': 41.88, 'RMSE': 65.56, 'MAPE': 24.61}

MAE Improvement: 0.55%

RMSE Improvement: 0.29%

MAPE Improvement: 0.12%

In [61]:
# Display feature importance
feature_importance: np.ndarray = final_rf_model.feature_importances_
feature_names: list[str] = train_df.drop("target").columns

# Get top 10 most important features
importance_df = pl.DataFrame(
    {"feature": feature_names, "importance": feature_importance}
).sort("importance", descending=True)

importance_df.head(15)

feature,importance
str,f64
"""current""",0.686207
"""cnt_lag_23""",0.117742
"""cnt_diff_1hr""",0.066908
"""cnt_diff_24hr""",0.018955
"""is_working_hour""",0.012579
"""sin_hour""",0.01227
"""cos_hour""",0.009187
"""cnt_lag_1""",0.0064
"""hr_lag_1""",0.006371


In [ ]:
metrics_base_with_advanced_feats

{'MAE': 41.65, 'RMSE': 65.37, 'MAPE': 24.58}

In [63]:
plot_actual_vs_predicted(
    y_true=y_test,
    y_pred=y_test_pred_optimized,
    n=400,
    title="Actual vs Predicted Bike Rental Counts (Optimized Model)",
)

Using first 400 values for plotting.


# Train An XGBoost Model

In [64]:
import xgboost as xgb

In [65]:
xgb_params: dict[str, Any] = {
    "objective": "reg:squarederror",
    "n_estimators": 100,
    "max_depth": 6,
    "learning_rate": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "early_stopping_rounds": 50,
    "random_state": 42,
}


n_splits: int = 5
test_size: int = 168  # 1 week of hourly data

tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size, gap=0)
xgb_reg = xgb.XGBRegressor(**xgb_params)
all_rmse: list[float] = []
all_mae: list[float] = []
all_mape: list[float] = []
print(tscv)

for i, (train_index, test_index) in enumerate(tscv.split(x_train), start=1):
    print(f"Fold {i}:")
    x_tr, x_val = x_train[train_index], x_train[test_index]
    y_tr, y_val = y_train[train_index], y_train[test_index]
    # Train the model
    xgb_reg.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], verbose=False)
    # Evaluate the model
    y_pred = xgb_reg.predict(x_val)
    metrics = compute_metrics(y_val, y_pred)
    print(f"Validation Metrics: {metrics}")
    all_rmse.append(metrics.get("RMSE"))
    all_mae.append(metrics.get("MAE"))
    all_mape.append(metrics.get("MAPE"))

print("\nCross-Validation Results:")
print(f"Average MAE over {n_splits} folds: {np.mean(all_mae).round(2)}")
print(f"Average RMSE over {n_splits} folds: {np.mean(all_rmse).round(2)}")
print(f"Average MAPE over {n_splits} folds: {np.mean(all_mape).round(2)}")

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=168)
Fold 1:
Validation Metrics: {'MAE': 32.77, 'RMSE': 53.28, 'MAPE': 27.09}
Fold 2:
Validation Metrics: {'MAE': 55.27, 'RMSE': 84.75, 'MAPE': 26.34}
Fold 3:
Validation Metrics: {'MAE': 33.52, 'RMSE': 54.69, 'MAPE': 29.79}
Fold 4:
Validation Metrics: {'MAE': 36.6, 'RMSE': 57.82, 'MAPE': 29.2}
Fold 5:
Validation Metrics: {'MAE': 32.67, 'RMSE': 53.24, 'MAPE': 26.34}

Cross-Validation Results:
Average MAE over 5 folds: 38.17
Average RMSE over 5 folds: 60.76
Average MAPE over 5 folds: 27.75


In [66]:
# Evaluate on the test set
y_test_pred_xgb = xgb_reg.predict(x_test)
metrics_xgb = compute_metrics(y_test, y_test_pred_xgb)
print(f"\nTest Set Metrics: {metrics_xgb}")


Test Set Metrics: {'MAE': 41.91, 'RMSE': 64.7, 'MAPE': 25.76}


In [67]:
plot_actual_vs_predicted(
    y_true=y_test,
    y_pred=y_test_pred_xgb,
    n=400,
    title="Actual vs Predicted Bike Rental Counts (XGBoost Model)",
)

Using first 400 values for plotting.


In [68]:
mae_improvement: float = pct_change(
    old=metrics_base_with_advanced_feats.get("MAE", 0),
    new=metrics_xgb.get("MAE", 0),
)
rmse_improvement: float = pct_change(
    old=metrics_base_with_advanced_feats.get("RMSE", 0),
    new=metrics_xgb.get("RMSE", 0),
)
mape_improvement: float = pct_change(
    old=metrics_base_with_advanced_feats.get("MAPE", 0),
    new=metrics_xgb.get("MAPE", 0),
)

console.print(f"MAE Improvement: {mae_improvement:.2f}%", style="highlight")
console.print(f"RMSE Improvement: {rmse_improvement:.2f}%", style="highlight")
console.print(f"MAPE Improvement: {mape_improvement:.2f}%", style="highlight")

MAE Improvement: 0.62%

RMSE Improvement: -1.02%

MAPE Improvement: 4.80%